In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np


In [2]:
path = tf.keras.utils.get_file("shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
text = open(path, 'rb').read().decode(encoding='utf-8').lower()

# Use only first 500,000 characters to make it faster
text = text[:500000]

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

# Step 3: Prepare sequences
seq_length = 60
step = 3  # faster dataset creation
sentences, next_chars = [], []
for i in range(0, len(text) - seq_length, step):
    sentences.append(text[i: i + seq_length])
    next_chars.append(text[i + seq_length])

# Vectorize data (faster using float32)
x = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.float32)
y = np.zeros((len(sentences), len(chars)), dtype=np.float32)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_idx[char]] = 1.0
    y[i, char_to_idx[next_chars[i]]] = 1.0


In [4]:
# Step 4: Build simple LSTM model
model = models.Sequential([
    layers.LSTM(128, input_shape=(seq_length, len(chars))),
    layers.Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
model.fit(x, y, batch_size=128, epochs=5)

Epoch 1/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 205s 156ms/step - loss: 2.7918
Epoch 2/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 202s 155ms/step - loss: 2.1761
Epoch 3/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 262s 156ms/step - loss: 2.0024
Epoch 4/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 203s 156ms/step - loss: 1.8859
Epoch 5/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 201s 154ms/step - loss: 1.8087


In [6]:
def generate_text(seed_text, length=200, temperature=0.5):
    generated = seed_text
    for _ in range(length):
        sampled = np.zeros((1, seq_length, len(chars)), dtype=np.float32)
        for t, char in enumerate(seed_text[-seq_length:]):
            if char in char_to_idx:
                sampled[0, t, char_to_idx[char]] = 1.0

        preds = model.predict(sampled, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_idx = np.random.choice(len(chars), p=preds)
        next_char = idx_to_char[next_idx]

        generated += next_char
        seed_text += next_char
    return generated


In [7]:
print("\n--- Generated Text (RNN) ---")
print(generate_text("to be or not to be ", length=300, temperature=0.6))


--- Generated Text (RNN) ---
to be or not to be ttee sooticeiotaitttyioti,asydpidifeo:ainge, is be so brought:
no more stoo to that fould dise i'll dis to treat
for what for be but you, go then lond ti sures,
and him tum would a dight and hear heart
and the say is not the sigr of his lome.

sicinius:
i we tree and grave the with sie my rings:
and
